In [1]:
from pyspark.sql import SparkSession, functions as f

In [2]:
spark = (
    SparkSession.builder.appName("Preprocessing_Yellow")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .config('spark.driver.memory', '3g')   
    .config('spark.executor.memory', '4g')  
    .config('spark.executor.instances', '2')  
    .config('spark.executor.cores', '2')
    .getOrCreate()
) 

your 131072x1 screen size is bogus. expect trouble
23/09/15 14:48:19 WARN Utils: Your hostname, NivethanAsus resolves to a loopback address: 127.0.1.1; using 172.23.155.90 instead (on interface eth0)
23/09/15 14:48:19 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/15 14:48:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
descriptions = spark.read.parquet("../../../data/insights/pre_insights/descriptions.parquet/")

In [4]:
tags_count = descriptions.groupBy("merchant_abn").agg(f.countDistinct("merchant_description").alias("description_count"))
print(tags_count.filter(tags_count.description_count > 1).count())

0


In [5]:
descriptions_agg = descriptions.groupBy("merchant_abn").\
            agg(f.first("merchant_description").alias("merchant_description"))

In [6]:
descriptions_count = descriptions_agg.groupBy("merchant_description").agg(f.count("merchant_abn").alias("number_of_merchants_with_description"))

In [7]:
descriptions_count.orderBy("number_of_merchants_with_description").count()

25

In [8]:
descriptions_agg.write.mode("overwrite").parquet("../../../data/insights/agg_insight_data/descriptions_agg.parquet")

In [9]:
spark.stop()

### Insights
- There 25 different descriptions for all merchants